## 0. Importing all necessary libs and tools

In [58]:
import torch
from ultralytics import YOLO
import os
import time
import sys
for module_name in list(sys.modules.keys()):
    if module_name.startswith('src.'):
        del sys.modules[module_name]
from src.config import MODEL_NAME, OVERLAPPING_PERCENTAGE, SLICE_SIZE, SLICES_FOLDER, PREDICT_FOLDER_PREFIX, CONFIDENCE_THRESHOLD, IOU_FOLDER_PREFIX, OUTLIER_FILTER_FOLDER_PREFIX, OUTLIER_THRESHOLD_K, IOU_THRESHOLD
print(MODEL_NAME, SLICE_SIZE, SLICES_FOLDER, PREDICT_FOLDER_PREFIX, CONFIDENCE_THRESHOLD, IOU_FOLDER_PREFIX, OUTLIER_FILTER_FOLDER_PREFIX, OUTLIER_THRESHOLD_K, IOU_THRESHOLD)
from src.model import get_devices, get_models, process_folders, read_annotation_file, create_annotation_file, read_annotation_folder
from src.tools.utils import get_destination_folder, create_destination_folder, get_slice_coordinates, copyfile
from src.tools.image_slicer import create_image_slices
from src.tools.iou_filter import filter_iou


best_4 640 slices predicted_images_with_annotations 0.4 iou_filtered outlier_filtered 3 0.5


## 1. Slicing image process


In [ ]:
images_folder = "dataset/0.0.1/АФС для обработки ИИ"
image_paths = [f"{images_folder}/{image_path}" for image_path in os.listdir(images_folder)]
for image_path in image_paths:
    create_image_slices(
        image_path=image_path, 
        overlap_percentage=OVERLAPPING_PERCENTAGE, 
        destination_folder=SLICES_FOLDER,
        slice_size=SLICE_SIZE
    )

## 1.2 Parallel slicing approach

In [ ]:
from src.tools.image_slicer import create_images_slices_parallel
create_images_slices_parallel()

Processing 211 images using 20 processes


Processing images: 100%|██████████| 211/211 [00:59<00:00,  3.55it/s]


Completed: 211 images, 37136 total slices


## 2. Processing slices with yolo model 

In [ ]:
devices = get_devices()
models = get_models(MODEL_NAME, devices)
print(f"Using devices: {devices}")
folder_name = get_destination_folder([PREDICT_FOLDER_PREFIX, MODEL_NAME, CONFIDENCE_THRESHOLD, "2"])
output_folder = create_destination_folder(folder_name)
total_detections = process_folders(models[1], devices[1], output_folder)

CUDA доступна: True
Название GPU: NVIDIA GeForce RTX 5090
Количество GPU: 2
Текущий GPU device: 0
Доступные устройства: ['cuda:0', 'cuda:1']
Модель загружена на: cuda:0
Модель загружена на: cuda:1
Using devices: ['cuda:0', 'cuda:1']
Обработка изображений на устройстве: cuda:1
пПапка вывода: predicted_images_with_annotations-best_4-0.4-2


## 2.2 Processing folders parallel

In [ ]:
devices = get_devices()
models = get_models(MODEL_NAME, devices)
folder_name = get_destination_folder([PREDICT_FOLDER_PREFIX, MODEL_NAME, CONFIDENCE_THRESHOLD, "2"])
output_folder = create_destination_folder(folder_name)
total_detections = process_folders_parallel(models, devices, output_folder)

CUDA доступна: True
Название GPU: NVIDIA GeForce RTX 5090
Количество GPU: 2
Текущий GPU device: 0
Доступные устройства: ['cuda:0', 'cuda:1']
Модель загружена на: cuda:0
Модель загружена на: cuda:1
Обработка 211 папок с детекциями используя 2 процессов


Processing folders:   0%|          | 0/211 [00:00<?, ?it/s]

## 3. Removing duplicates using IOU coefficient  

In [ ]:
source_folder = get_destination_folder([PREDICT_FOLDER_PREFIX, MODEL_NAME, CONFIDENCE_THRESHOLD, "2"])
boxes_list = read_annotation_folder(source_folder)
folder_name = get_destination_folder([IOU_FOLDER_PREFIX, MODEL_NAME, IOU_THRESHOLD])
output_folder = create_destination_folder(folder_name)
filtered_boxes_list = filter_iou(boxes_list)

Всего боксов для сравнения: 2304
intersection_area 753.1552005806006
union_area 753.4952336396091
Боксы с IoU > 0.5:
Бокс 1: [571.717041015625, 462.65380859375, 595.793701171875, 493.9422607421875] (из predicted_images_with_annotations-best_4-0.4-2/2025_08_05_PhotoRieboR4_g201b201078_f003_090/2025_08_05_PhotoRieboR4_g201b201078_f003_090_slice_000_000_0_0.png)
Бокс 2: [571.7189254760742, 462.65179443359375, 595.7977066040039, 493.93780517578125] (из predicted_images_with_annotations-best_4-0.4-2/2025_08_05_PhotoRieboR4_g201b201078_f003_090/2025_08_05_PhotoRieboR4_g201b201078_f003_090_slice_000_001_512_0.png)
IoU: 0.9995
---
intersection_area 823.5333597064018
union_area 825.2439309265465
Боксы с IoU > 0.5:
Бокс 1: [546.153076171875, 282.1562194824219, 566.991943359375, 321.6753234863281] (из predicted_images_with_annotations-best_4-0.4-2/2025_08_05_PhotoRieboR4_g201b201078_f003_090/2025_08_05_PhotoRieboR4_g201b201078_f003_090_slice_000_000_0_0.png)
Бокс 2: [546.1235046386719, 282.147369

In [ ]:
original_images_folder = "dataset/0.0.1/АФС для обработки ИИ/"

for image_path in os.listdir(original_images_folder):
    if image_path.endswith(".png"):
        source_image_path = f"{original_images_folder}/{image_path}"
        destination_image_path = f"{output_folder}/{image_path}"
        copyfile(source_image_path, destination_image_path)
        
        with Image.open(source_image_path) as img:
            width, height = img.size
        
        matching_boxes = [box for box_group in filtered_boxes_list 
                         if box_group['source_image_path'].endswith(image_path) 
                         for box in box_group['coordinates']]
        
        create_annotation_file(
            image_path=destination_image_path,
            boxes=matching_boxes,
            classes_folder=output_folder,
            image_size=(width, height)
        )


In [ ]:
predicted_images_path = "predicted_images_with_annotations-best_4-0.4-2"
count = 0
for folder in os.listdir(predicted_images_path):
    folder_path = f"{predicted_images_path}/{folder}"
    for file in os.listdir(folder_path):
        if file.endswith(".txt") and not file.startswith("classes"):
            file_path = f"{folder_path}/{file}"
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
            print(f"File: {file_path}, Boxes: {len(lines)}")
            count += len(lines)
print(f"Total boxes: {count}")